In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open('/content/sample_data/dog.jpg')
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

tensor([ 4.0412e-01, -2.2594e+00, -2.8336e+00, -3.9217e+00, -8.1416e-01,
        -3.7446e-01, -4.3653e+00, -2.3753e+00, -1.6658e+00, -1.0652e+00,
         9.9271e-01, -2.1959e+00, -1.5200e+00, -2.3526e+00, -2.9149e+00,
        -3.5916e+00, -9.1004e-01, -2.5952e-01, -2.0446e+00, -1.7821e+00,
        -4.0761e+00, -1.3370e-01, -2.7504e+00,  7.4692e-02, -7.1244e-01,
        -1.4895e+00, -1.1990e+00, -3.4583e+00, -1.6337e+00, -1.3211e+00,
        -2.9910e+00, -3.2066e+00, -9.4650e-01,  1.4657e+00,  9.9741e-01,
         2.1641e+00,  2.3657e+00,  5.9431e-01,  2.0586e-01, -1.2888e+00,
        -4.0173e+00, -9.7619e-01, -5.2934e+00, -9.7621e-01, -1.4298e+00,
         2.6997e+00, -3.3679e+00, -5.5329e-01, -1.1845e+00, -2.6174e+00,
        -1.3527e+00,  1.8589e+00,  1.2477e-01, -1.0625e+00, -2.6410e-01,
        -3.9199e+00, -2.1368e+00, -3.4435e+00, -3.7944e+00, -1.5277e+00,
        -6.5748e-01, -9.7708e-01, -1.5995e+00,  1.3149e+00, -4.4680e-01,
        -2.1600e+00, -6.8226e-01, -1.0704e+00, -1.2

In [3]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2022-04-11 12:30:40--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2022-04-11 12:30:40 (76.4 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [4]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

pug 0.9200333952903748
Brabancon griffon 0.06315470486879349
bull mastiff 0.009937530383467674
French bulldog 0.004279797431081533
Pekinese 0.0006088711670599878
